In [1]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import xgboost
import random
import datetime
import csv

from xgboost.sklearn import XGBRegressor
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import explained_variance_score

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 업로드를 통해 파일 불러오기
from google.colab import files
uploaded = files.upload()

Saving Total_air.csv to Total_air.csv


In [144]:
df = pd.read_csv('Fukuoka_air_전처리전.csv')

In [145]:
df

,month,day,air_class,airline_name,departure_time,departure_airport,arrival_time,arrival_airport,duration,price
0,7,22,비즈니스석,여러항공사,18:55,FUK,13:15,GMP,"경유 1, 18시간 20분","834,600"
1,7,22,비즈니스석,여러항공사,07:00,FUK,22:25,GMP,"경유 1, 15시간 25분","861,000"
2,7,22,비즈니스석,여러항공사,15:55,FUK,22:25,GMP,"경유 1, 06시간 30분","861,000"
3,7,22,비즈니스석,여러항공사,07:50,FUK,22:25,GMP,"경유 1, 14시간 35분","861,000"
4,7,22,비즈니스석,여러항공사,19:10,FUK,13:15,GMP,"경유 1, 18시간 05분","902,800"
...,...,...,...,...,...,...,...,...,...,...
21354,8,4,프리미엄 일반석,캐세이퍼시픽항공,13:35,ICN,12:50,FUK,"경유 1, 23시간 15분","4,312,200"
21355,8,4,프리미엄 일반석,캐세이퍼시픽항공,15:10,ICN,15:30,FUK,"경유 1, 24시간 20분","4,312,200"
21356,8,4,프리미엄 일반석,캐세이퍼시픽항공,13:35,ICN,15:30,FUK,"경유 1, 25시간 55분","4,312,200"
21357,8,4,프리미엄 일반석,캐세이퍼시픽항공,10:10,ICN,12:50,FUK,"경유 1, 26시간 40분","4,312,200"


* 크롤링 데이터 전처리

In [147]:
df_test = df.copy()
# ,를 기준으로 'direct_flight', 'duration' 나눔
df_test[['direct_flight', 'duration']] = df_test['duration'].str.split(',', 1, expand=True)
# direct_flight를 직항이면 1 아니면 0으로 변환
df_test['direct_flight'] = df_test['direct_flight'].apply(lambda x: 1 if x.strip() == '직항' else 0)
df_test['direct_flight'] = df_test['direct_flight'].astype('category')
# price 데이터에서 , 빼기
df_test['price'] = df_test['price'].str.replace(',', '').astype(int)
# departure_time, arrival_time 데이터를 TIME 데이터 타입으로 변경
df_test['departure_time'] = pd.to_datetime(df_test['departure_time']).dt.time
df_test['arrival_time'] = pd.to_datetime(df_test['arrival_time']).dt.time

# duration데이터 TIME 형식으로 변환
df_test[['hour', 'minute']] = df_test['duration'].str.extract(r'(\d+)시간 (\d+)분').astype(int)
df_test['duration'] = pd.to_timedelta(df_test['hour'] * 60 + df_test['minute'], unit='m')

# 불필요한 칼럼 삭제
df_test.drop(columns=['hour', 'minute'], inplace=True)

# '0 days' 부분을 없애고 시간 형식으로 변환
df_test['duration'] = df_test['duration'].astype(str).str.split().str[-1]

# 'month'와 'day' 칼럼 값을 문자열로 합친 뒤, 날짜 형식으로 변환
df_test['date'] = pd.to_datetime('2023-' + df_test['month'].astype(str) + '-' + df_test['day'].astype(str))
df_test.drop(columns=['month', 'day'], inplace=True)

# 칼럼 순서 변경
df_test = df_test[['date', 'air_class', 'airline_name', 'departure_time', 'departure_airport', 'arrival_time',
         'arrival_airport', 'direct_flight', 'duration', 'price']]

In [148]:
df_test

,date,air_class,airline_name,departure_time,departure_airport,arrival_time,arrival_airport,direct_flight,duration,price
0,2023-07-22,비즈니스석,여러항공사,18:55:00,FUK,13:15:00,GMP,0,18:20:00,834600
1,2023-07-22,비즈니스석,여러항공사,07:00:00,FUK,22:25:00,GMP,0,15:25:00,861000
2,2023-07-22,비즈니스석,여러항공사,15:55:00,FUK,22:25:00,GMP,0,06:30:00,861000
3,2023-07-22,비즈니스석,여러항공사,07:50:00,FUK,22:25:00,GMP,0,14:35:00,861000
4,2023-07-22,비즈니스석,여러항공사,19:10:00,FUK,13:15:00,GMP,0,18:05:00,902800
...,...,...,...,...,...,...,...,...,...,...
21354,2023-08-04,프리미엄 일반석,캐세이퍼시픽항공,13:35:00,ICN,12:50:00,FUK,0,23:15:00,4312200
21355,2023-08-04,프리미엄 일반석,캐세이퍼시픽항공,15:10:00,ICN,15:30:00,FUK,0,00:20:00,4312200
21356,2023-08-04,프리미엄 일반석,캐세이퍼시픽항공,13:35:00,ICN,15:30:00,FUK,0,01:55:00,4312200
21357,2023-08-04,프리미엄 일반석,캐세이퍼시픽항공,10:10:00,ICN,12:50:00,FUK,0,02:40:00,4312200


* 통합 항공 데이터 예측 모델 만들기

In [4]:
df_original = pd.read_csv('Total_air.csv')

In [5]:
df_original = df_original.drop('Unnamed: 0', axis=1)
df_original

,date,air_class,airline_name,departure_time,departure_airport,arrival_time,arrival_airport,direct_flight,duration,price,region
0,2023-07-22,일반석,에어부산,15:20:00,ICN,17:10:00,KIX,1,01:50:00,138817,오사카
1,2023-07-22,일반석,에어부산,13:20:00,ICN,15:10:00,KIX,1,01:50:00,138817,오사카
2,2023-07-22,일반석,에어서울,13:15:00,ICN,15:15:00,KIX,1,02:00:00,162300,오사카
3,2023-07-22,일반석,진에어,16:10:00,ICN,18:00:00,KIX,1,01:50:00,162700,오사카
4,2023-07-22,일반석,제주항공,13:30:00,ICN,15:20:00,KIX,1,01:50:00,176700,오사카
...,...,...,...,...,...,...,...,...,...,...,...
46766,2023-08-04,일등석,ANA,19:00:00,HND,13:00:00,ICN,0,18:00:00,2056400,도쿄
46767,2023-08-04,일등석,ANA,08:30:00,HND,15:25:00,ICN,0,06:55:00,2703600,도쿄
46768,2023-08-04,일등석,ANA,07:30:00,HND,15:25:00,ICN,0,07:55:00,2703600,도쿄
46769,2023-08-04,일등석,ANA,06:15:00,HND,15:25:00,ICN,0,09:10:00,2703600,도쿄


In [6]:
# 필터링 조건에 해당하는 데이터 추출
filtered_data = df_original[(df_original['departure_airport'] == 'NRT') & (df_original['arrival_airport'] == 'GMP')]

# 대한항공이고 프리미엄 일반석인 데이터 추출
desired_data = filtered_data[(filtered_data['airline_name'] == '대한항공') & (filtered_data['air_class'] == '프리미엄 일반석')]

# 결과 출력
print(desired_data)

             date air_class airline_name departure_time departure_airport  \
43590  2023-07-31  프리미엄 일반석         대한항공       19:20:00               NRT   
43591  2023-07-31  프리미엄 일반석         대한항공       19:20:00               NRT   
43743  2023-08-01  프리미엄 일반석         대한항공       12:45:00               NRT   
43744  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   
43745  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   
43773  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   
43774  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   
43775  2023-08-01  프리미엄 일반석         대한항공       12:45:00               NRT   
43776  2023-08-01  프리미엄 일반석         대한항공       12:45:00               NRT   
43777  2023-08-01  프리미엄 일반석         대한항공       12:45:00               NRT   
43778  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   
43779  2023-08-01  프리미엄 일반석         대한항공       19:20:00               NRT   

* 루트별 항공사 이름 구하기

In [14]:
filtered_df = df_original[(df_original['departure_airport'] == 'GMP') & (df_original['arrival_airport'] == 'FUK')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['일본항공' '여러항공사' 'ANA' '중국국제항공' '중국동방항공' '에바항공' '에어부산' 'ANA항공' '제주항공']


In [15]:
filtered_df = df_original[(df_original['departure_airport'] == 'GMP') & (df_original['arrival_airport'] == 'HND')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['ANA' '아시아나항공' '여러항공사' '일본항공' '대한항공' '상하이항공' '중국동방항공' '중국국제항공' 'ANA항공'
 '상해항공' '에바항공']


In [16]:
filtered_df = df_original[(df_original['departure_airport'] == 'GMP') & (df_original['arrival_airport'] == 'KIX')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['아시아나항공' '제주항공' '한에어' '대한항공' '일본항공' 'ANA' '에어부산' '중국남방항공' '여러항공사'
 '중국국제항공' '상하이항공' '중국동방항공' '에바항공' 'ANA항공' '상해항공']


In [22]:
filtered_df = df_original[(df_original['departure_airport'] == 'GMP') & (df_original['arrival_airport'] == 'NRT')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['대한항공' '여러항공사' '일본항공' '에어부산' 'ANA' '상하이항공' '중국동방항공' '제주항공' 'ANA항공' '상해항공'
 '중국국제항공' '에바항공']


In [23]:
filtered_df = df_original[(df_original['departure_airport'] == 'ICN') & (df_original['arrival_airport'] == 'FUK')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['에어서울' '티웨이항공' '제주항공' '진에어' '아시아나항공' '대한항공' '일본항공' 'ANA' '여러항공사' '한에어'
 '베트남항공' '중국동방항공' '필리핀항공' '에바항공' '캐세이퍼시픽항공' '비엣젯항공' '홍콩항공' '에어부산' '중국국제항공'
 '홍콩익스프레스항공' '싱가포르항공' 'ANA항공']


In [24]:
filtered_df = df_original[(df_original['departure_airport'] == 'ICN') & (df_original['arrival_airport'] == 'HND')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['대한항공' '피치항공' '여러항공사' '일본항공' 'ANA' '비엣젯항공' '중국동방항공' '중국국제항공' '중국남방항공'
 '상하이항공' '에바항공' '필리핀항공' '캐세이퍼시픽항공' '싱가포르항공' '아시아나항공' 'ANA항공' '홍콩익스프레스'
 '상해항공']


In [25]:
filtered_df = df_original[(df_original['departure_airport'] == 'ICN') & (df_original['arrival_airport'] == 'KIX')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['에어부산' '에어서울' '진에어' '제주항공' '티웨이항공' '아시아나항공' '피치항공' '한에어' '대한항공' '일본항공'
 'ANA' '여러항공사' '중국남방항공' '비엣젯항공' '중국국제항공' '중국동방항공' '산둥항공' '홍콩항공' '미아트몽골항공'
 '상하이항공' '샤먼항공' '베트남항공' '말레이시아항공' '에바항공' '캐세이퍼시픽항공' '필리핀항공' '스쿠트항공'
 '에어캐나다' '쓰촨항공' 'ANA항공' '중국하문항공' '상해항공' '홍콩익스프레스항공']


In [26]:
filtered_df = df_original[(df_original['departure_airport'] == 'ICN') & (df_original['arrival_airport'] == 'NRT')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['에어서울' '에어부산' '제주항공' '에어프레미아' '한에어' '아시아나항공' '대한항공' 'ANA' 'ZIPAIR TOKYO'
 '일본항공' '여러항공사' '뱀부에어웨이스' '상하이항공' '홍콩항공' '필리핀항공' '베트남항공' '말레이시아항공'
 '중국동방항공' '에바항공' '중국국제항공' '캐세이퍼시픽항공' '중국남방항공' '진에어' '티웨이항공' '에티오피아항공'
 '비엣젯항공' 'ANA항공' '홍콩익스프레스' '상해항공' '미아트몽골항공' '산둥항공' '가루다인도네시아항공' '마카오항공'
 '몽골항공']


In [18]:
filtered_df = df_original[(df_original['departure_airport'] == 'FUK') & (df_original['arrival_airport'] == 'GMP')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['여러항공사' '일본항공' 'ANA' '중국동방항공' '중국국제항공' '제주항공' '에어부산' '아시아나항공' '에바항공'
 'ANA항공']


In [19]:
filtered_df = df_original[(df_original['departure_airport'] == 'HND') & (df_original['arrival_airport'] == 'GMP')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['아시아나항공' '대한항공' '여러항공사' '중국동방항공' '중국국제항공' '일본항공' '중국남방항공' '상하이항공' 'ANA'
 '에바항공' '중화항공' 'ANA항공']


In [20]:
filtered_df = df_original[(df_original['departure_airport'] == 'KIX') & (df_original['arrival_airport'] == 'GMP')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['제주항공' '한에어' '아시아나항공' '대한항공' '에어부산' '중국동방항공' '여러항공사' '상하이항공' '중국국제항공'
 '일본항공' 'ANA' '중화항공' '중국남방항공' '상해항공' '에바항공']


In [21]:
filtered_df = df_original[(df_original['departure_airport'] == 'NRT') & (df_original['arrival_airport'] == 'GMP')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['제주항공' '여러항공사' '에어부산' '대한항공' '중국국제항공' '일본항공' '중국동방항공' '중국남방항공' 'ANA'
 '상하이항공' '중화항공' '에바항공' '상해항공']


In [27]:
filtered_df = df_original[(df_original['departure_airport'] == 'FUK') & (df_original['arrival_airport'] == 'ICN')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['아시아나항공' '대한항공' '여러항공사' '일본항공' 'ANA' '중화항공' '에바항공' '베트남항공' '필리핀항공'
 '중국동방항공' '캐세이퍼시픽항공' '타이항공' '중국국제항공' '제주항공' '티웨이항공' '에어서울' '에어부산' '한에어'
 '진에어' '비엣젯항공' '홍콩익스프레스항공' '홍콩항공' 'ANA항공' '싱가포르항공']


In [28]:
filtered_df = df_original[(df_original['departure_airport'] == 'HND') & (df_original['arrival_airport'] == 'ICN')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['여러항공사' '비엣젯항공' '대한항공' '중국국제항공' '중국동방항공' '중국남방항공' '상하이항공' '일본항공' '피치항공'
 '가루다인도네시아항공' '필리핀항공' '중화항공' 'ANA' '베트남항공' '에바항공' '캐세이퍼시픽항공' '천진항공'
 '아시아나항공' '톈진항공' 'ANA항공' '말레이시아항공' '타이항공' '싱가포르항공']


In [29]:
filtered_df = df_original[(df_original['departure_airport'] == 'KIX') & (df_original['arrival_airport'] == 'ICN')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['티웨이항공' '에어서울' '진에어' '제주항공' '에어부산' '피치항공' '한에어' '대한항공' '아시아나항공' '중국동방항공'
 '비엣젯항공' '중국국제항공' '여러항공사' '상하이항공' '베트남항공' '산둥항공' '샤먼항공' '중국남방항공' '미아트몽골항공'
 '중화항공' '일본항공' '필리핀항공' 'ANA' '에바항공' '캐세이퍼시픽항공' '말레이시아항공' '타이항공' '싱가포르항공'
 '스쿠트항공' '바틱에어 말레이시아' '홍콩항공' '상해항공' '쓰촨항공' '톈진항공' '중국하문항공' '홍콩익스프레스항공'
 '에어캐나다' '마카오항공' '선전항공' '루프트한자']


In [30]:
filtered_df = df_original[(df_original['departure_airport'] == 'NRT') & (df_original['arrival_airport'] == 'ICN')]

# Get the unique values of the 'airline_name' column
airline_names = filtered_df['airline_name'].unique()
print(airline_names)

['티웨이항공' '진에어' '에어프레미아' '에어서울' '제주항공' '에어부산' '한에어' '대한항공' 'ZIPAIR TOKYO'
 '아시아나항공' '여러항공사' '비엣젯항공' '중국국제항공' '미아트몽골항공' '뱀부에어웨이스' '중국남방항공' '중화항공'
 '일본항공' '선전항공' '베트남항공' '중국동방항공' '홍콩익스프레스항공' '마카오항공' '가루다인도네시아항공' '필리핀항공'
 'ANA' '에바항공' '캐세이퍼시픽항공' '말레이시아항공' '상하이항공' '타이항공' '싱가포르항공' '에티오피아항공'
 '홍콩항공' '몽골항공' '바틱에어 말레이시아' '스쿠트항공' '상해항공' '뱀부항공' '에어캐나다' '에어인디아']


In [7]:
df = df_original.copy()

In [8]:
# duration 칼럼의 값을 분으로 변환하는 함수
def convert_duration_to_minutes(duration_str):
    time_components = duration_str.split(':')
    hours = int(time_components[0])
    minutes = int(time_components[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

# 데이터프레임에서 duration 칼럼을 분으로 변환
df['duration'] = df['duration'].apply(convert_duration_to_minutes)
df

,date,air_class,airline_name,departure_time,departure_airport,arrival_time,arrival_airport,direct_flight,duration,price,region
0,2023-07-22,일반석,에어부산,15:20:00,ICN,17:10:00,KIX,1,110,138817,오사카
1,2023-07-22,일반석,에어부산,13:20:00,ICN,15:10:00,KIX,1,110,138817,오사카
2,2023-07-22,일반석,에어서울,13:15:00,ICN,15:15:00,KIX,1,120,162300,오사카
3,2023-07-22,일반석,진에어,16:10:00,ICN,18:00:00,KIX,1,110,162700,오사카
4,2023-07-22,일반석,제주항공,13:30:00,ICN,15:20:00,KIX,1,110,176700,오사카
...,...,...,...,...,...,...,...,...,...,...,...
46766,2023-08-04,일등석,ANA,19:00:00,HND,13:00:00,ICN,0,1080,2056400,도쿄
46767,2023-08-04,일등석,ANA,08:30:00,HND,15:25:00,ICN,0,415,2703600,도쿄
46768,2023-08-04,일등석,ANA,07:30:00,HND,15:25:00,ICN,0,475,2703600,도쿄
46769,2023-08-04,일등석,ANA,06:15:00,HND,15:25:00,ICN,0,550,2703600,도쿄


In [53]:
df['airline_name'].unique()

array(['에어부산', '에어서울', '진에어', '제주항공', '티웨이항공', '아시아나항공', '피치항공', '한에어',
       '대한항공', '일본항공', 'ANA', '여러항공사', '중국남방항공', '비엣젯항공', '중국국제항공',
       '상하이항공', '중국동방항공', '산둥항공', '홍콩항공', '미아트몽골항공', '샤먼항공', '베트남항공',
       '말레이시아항공', '에바항공', '캐세이퍼시픽항공', '필리핀항공', '스쿠트항공', '에어캐나다', '쓰촨항공',
       'ANA항공', '중국하문항공', '상해항공', '홍콩익스프레스항공', '중화항공', '타이항공', '싱가포르항공',
       '바틱에어 말레이시아', '톈진항공', '마카오항공', '선전항공', '루프트한자', '에어프레미아',
       'ZIPAIR TOKYO', '뱀부에어웨이스', '에티오피아항공', '홍콩익스프레스', '가루다인도네시아항공',
       '몽골항공', '천진항공', '뱀부항공', '에어인디아'], dtype=object)

In [87]:
df['departure_airport'].unique()

array(['ICN', 'GMP', 'KIX', 'FUK', 'NRT', 'HND'], dtype=object)

In [9]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop('date', axis=1)
df

,air_class,airline_name,departure_time,departure_airport,arrival_time,arrival_airport,direct_flight,duration,price,region,year,month,day
0,일반석,에어부산,15:20:00,ICN,17:10:00,KIX,1,110,138817,오사카,2023,7,22
1,일반석,에어부산,13:20:00,ICN,15:10:00,KIX,1,110,138817,오사카,2023,7,22
2,일반석,에어서울,13:15:00,ICN,15:15:00,KIX,1,120,162300,오사카,2023,7,22
3,일반석,진에어,16:10:00,ICN,18:00:00,KIX,1,110,162700,오사카,2023,7,22
4,일반석,제주항공,13:30:00,ICN,15:20:00,KIX,1,110,176700,오사카,2023,7,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46766,일등석,ANA,19:00:00,HND,13:00:00,ICN,0,1080,2056400,도쿄,2023,8,4
46767,일등석,ANA,08:30:00,HND,15:25:00,ICN,0,415,2703600,도쿄,2023,8,4
46768,일등석,ANA,07:30:00,HND,15:25:00,ICN,0,475,2703600,도쿄,2023,8,4
46769,일등석,ANA,06:15:00,HND,15:25:00,ICN,0,550,2703600,도쿄,2023,8,4


In [10]:
df = df.drop('departure_time', axis=1)
df = df.drop('arrival_time', axis=1)
#df = df.drop('duration', axis=1)
df

,air_class,airline_name,departure_airport,arrival_airport,direct_flight,duration,price,region,year,month,day
0,일반석,에어부산,ICN,KIX,1,110,138817,오사카,2023,7,22
1,일반석,에어부산,ICN,KIX,1,110,138817,오사카,2023,7,22
2,일반석,에어서울,ICN,KIX,1,120,162300,오사카,2023,7,22
3,일반석,진에어,ICN,KIX,1,110,162700,오사카,2023,7,22
4,일반석,제주항공,ICN,KIX,1,110,176700,오사카,2023,7,22
...,...,...,...,...,...,...,...,...,...,...,...
46766,일등석,ANA,HND,ICN,0,1080,2056400,도쿄,2023,8,4
46767,일등석,ANA,HND,ICN,0,415,2703600,도쿄,2023,8,4
46768,일등석,ANA,HND,ICN,0,475,2703600,도쿄,2023,8,4
46769,일등석,ANA,HND,ICN,0,550,2703600,도쿄,2023,8,4


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46771 entries, 0 to 46770
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   air_class          46771 non-null  object
 1   airline_name       46771 non-null  object
 2   departure_airport  46771 non-null  object
 3   arrival_airport    46771 non-null  object
 4   direct_flight      46771 non-null  int64 
 5   duration           46771 non-null  int64 
 6   price              46771 non-null  int64 
 7   region             46771 non-null  object
 8   year               46771 non-null  int64 
 9   month              46771 non-null  int64 
 10  day                46771 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.9+ MB


In [11]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.25, random_state=42)

In [37]:
# 파이프라인 구성
xgb_pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    XGBRegressor()
)

# 하이퍼파라미터 후보들을 지정합니다.
param_grid = {
    'xgbregressor__n_estimators': [200, 250, 300],  # n_estimators 후보값들
    'xgbregressor__max_depth': [6, 7, 8, 9],          # max_depth 후보값들
    'xgbregressor__learning_rate': [0.1, 0.2] # learning_rate 후보값들
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터를 찾습니다.
grid_search = GridSearchCV(xgb_pipe, param_grid, cv=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('simpleimputer',
                                        SimpleImputer(strategy='median')),
                                       ('xgbregressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_metric=None,
                                                     feature_ty...
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=None, ...))]),
             param_grid={'xgbregressor__learning_rate': [0.1, 0.2],
                         'xgbregressor__max_depth': [6, 7, 8, 9],
                         'xgbregressor__n_estimators': [200, 250, 300]})

In [38]:
# 최적의 하이퍼파라미터와 그 때의 점수를 확인합니다.
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# 최적의 하이퍼파라미터를 사용하여 모델을 재학습합니다.
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# 최적의 모델을 사용하여 예측합니다.
y_pred = best_model.predict(X_test)

Best Parameters: {'xgbregressor__learning_rate': 0.2, 'xgbregressor__max_depth': 6, 'xgbregressor__n_estimators': 300}
Best Score: 0.8310462996712084


In [39]:
predictions = best_model.predict(X_test)

In [40]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 실제값
y_real = y_test

predict = y_test.mean()

# 기준 모델
y_base = [predict]*len(y_test)


# mse, rmse, mae, r2를 출력하는 함수입니다.

def eval_models(y_pred, y_real=y_real) :
    mse = mean_squared_error(y_real, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)

    return mse, rmse, mae, r2

base_mse, base_rmse, base_mae, base_r2 = eval_models(y_base)
xgb_mse, xgb_rmse, xgb_mae, xgb_r2 = eval_models(predictions)

comparison_metrics = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base','xgb'])
comparison_metrics['Base'] = [base_mse, base_rmse, base_mae, base_r2 ]
comparison_metrics['xgb'] = [xgb_mse, xgb_rmse, xgb_mae, xgb_r2 ]
comparison_metrics

,Base,xgb
mse,6.113238e+11,9.240870e+10
rmse,7.818720e+05,3.039880e+05
mae,5.560733e+05,1.684599e+05
r2,0.000000e+00,8.488384e-01


In [41]:
# 데이터입력시 예측 테스트
data = {
    'air_class'	: ['일반석'],
    'airline_name' : ['에어서울'],
    'departure_airport' :	['ICN'],
    'arrival_airport'	: ['KIX'],
    'direct_flight'	: [1],
    #'duration' : ['110'],
    'region' : ['오사카'],
    'year' : [2023],
    'month'	: [7],
    'day' : [23]
}
test = pd.DataFrame(data)
prediction2 = best_model.predict(test)

data2 = {
    'air_class'	: ['일반석'],
    'airline_name' : ['에어서울'],
    'departure_airport' :	['KIX'],
    'arrival_airport'	: ['ICN'],
    'direct_flight'	: [1],
    #'duration' : ['110'],
    'region' : ['오사카'],
    'year' : [2023],
    'month'	: [7],
    'day' : [31]
}
test2 = pd.DataFrame(data2)
prediction3 = best_model.predict(test2)

print(prediction2)
print(prediction3)
print(prediction2 + prediction3)

[206167.92]
[338521.03]
[544688.94]


In [42]:
import joblib
joblib.dump(best_model, 'JP_air_model.pkl')

['JP_air_model.pkl']

* -------------------------------------------

모델 불러올 때

import joblib


model = joblib.load('apart_sale_xgb_model.pkl')

7월 22일부터 8월 4일 까지의 데이터 수집

duration 데이터에 대한 효용여부
사용하였을 때 더 성능이 좋았음

더 정확한 정보를 주는 것을 목표로 할 것인지, 아니면 '선호 여행 시간'이라는 인풋 퍼널을 제거함으로써 성능은 조금 낮을 수 있지만 더 편리한 서비스(가는날, 오는날만 입력하므로)를 제공할 것인지 결정

더 정확한 정보를 주는 것을 목표로 설정

날짜 데이터인 date 칼럼을 원핫 인코딩으로
year month day로 나누어서 학습

직항과 경유의 경우 걸리는 시간 데이터를 사용하였으므로 경유를 더 나누지는 않고 직항여부로 1과 0으로 전처리

In [1]:
pip list | grep scikit-learn

scikit-learn                     1.2.2
